In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from datetime import datetime
import time

def authenticate_google_drive():
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  
    drive = GoogleDrive(gauth)  
    return drive

def fetch_stock_data(ticker, start_date, end_date, retries=3):
    """Fetch stock data using yfinance with retries in case of failure."""
    for i in range(retries):
        try:
            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"No data fetched for {ticker}.")
            return data
        except Exception as e:
            print(f"Error fetching data for {ticker}, attempt {i + 1}/{retries}: {e}")
            if i < retries - 1:
                time.sleep(2)  
            else:
                return None

def calculate_sma(data, window=5):
    if 'Close' in data.columns:
        data['SMA_5'] = data['Close'].rolling(window=window).mean()
    else:
        print("No 'Close' column found in data.")
    return data

def plot_candlestick(data, ticker, filename):
    """Generate and save a candlestick chart with SMA to an HTML file."""
    if not all(col in data.columns for col in ['Open', 'High', 'Low', 'Close']):
        print(f"Data missing required fields for {ticker}. Skipping plot generation.")
        return

    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    if 'SMA_5' in data.columns:
        fig.add_trace(go.Scatter(x=data.index, y=data['SMA_5'], mode='lines', name='SMA 5'))

    fig.update_layout(title=f'Candlestick Chart with 5-period SMA for {ticker}',
                      xaxis_title='Date',
                      yaxis_title='Price')
    fig.write_html(filename)

def upload_to_drive(drive, file_path, title, retries=3):
    for i in range(retries):
        try:
            file1 = drive.CreateFile({'title': title})
            file1.SetContentFile(file_path)
            file1.Upload()
            print(f"Uploaded {title} successfully.")
            break
        except Exception as e:
            print(f"Error uploading {file_path} to Google Drive, attempt {i + 1}/{retries}: {e}")
            if i < retries - 1:
                time.sleep(2)
            else:
                print(f"Failed to upload {file_path} after {retries} attempts.")

def main():
    
    try:
        tickers = pd.read_csv('hmm.csv')['ticker'].dropna().tolist()
    except FileNotFoundError:
        print("File 'hmm.csv' not found. Ensure the file exists and has a 'ticker' column.")
        return
    except KeyError:
        print("The CSV file does not contain a 'ticker' column.")
        return

    start_date = '2023-01-01'
    end_date = '2024-01-01'

    
    drive = authenticate_google_drive()

    for ticker in tickers:
        print(f"Processing {ticker}...")

        stock_data = fetch_stock_data(ticker, start_date, end_date)
        
        if stock_data is not None and not stock_data.empty:
            stock_data = calculate_sma(stock_data)
            csv_filename = f'{ticker}_stock_data.csv'
            stock_data.to_csv(csv_filename)

            html_filename = f'{ticker}_candlestick_chart.html'
            plot_candlestick(stock_data, ticker, html_filename)

            
            upload_to_drive(drive, csv_filename, f'{ticker}_stock_data.csv')
            upload_to_drive(drive, html_filename, f'{ticker}_candlestick_chart.html')

            time.sleep(5)  
        else:
            print(f"Skipping {ticker} due to empty or failed stock data.")

if __name__ == "__main__":
    main()


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=847622577824-fti6ii4r4i58dno3o6q1ms70shukj5qa.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Processing AAPL...


[*********************100%***********************]  1 of 1 completed


Uploaded AAPL_stock_data.csv successfully.
Uploaded AAPL_candlestick_chart.html successfully.


[*********************100%***********************]  1 of 1 completed

Processing MSFT...


Uploaded MSFT_stock_data.csv successfully.
Uploaded MSFT_candlestick_chart.html successfully.
Processing TSLA...


[*********************100%***********************]  1 of 1 completed


Uploaded TSLA_stock_data.csv successfully.
Uploaded TSLA_candlestick_chart.html successfully.
Processing AMZN...


[*********************100%***********************]  1 of 1 completed


Uploaded AMZN_stock_data.csv successfully.
Uploaded AMZN_candlestick_chart.html successfully.
